In [1]:
# generate one path for n assets
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt 

from simulation import *

In [2]:
import torch
import torch.nn as nn

class SimpleLSTMModel(nn.Module):
    def __init__(self, hidden_size, output_size=1):  # Output size is set to 1 by default
        super(SimpleLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        # LSTM layer expects input of shape (batch_size, seq_length, features)
        # For a univariate sequence, features=1
        self.lstm = nn.LSTM(1, hidden_size, batch_first=True)
        # Fully connected layer to map the hidden state output to the desired output size
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Add a batch dimension and feature dimension to x
        # Reshaping x from (n,) to (1, n, 1) to fit LSTM input requirements
        x = x.unsqueeze(0).unsqueeze(-1)  # Now x is of shape [1, seq_length, 1]
        # Process x through the LSTM
        lstm_out, _ = self.lstm(x)
        # Only use the output from the last time step
        # This assumes you're interested in the final output for sequence processing
        out = self.fc(lstm_out[:, -1, :])
        # Squeeze the output to remove 1-dimensions, aiming for a scalar output
        return out.squeeze()

# Initialize the model
hidden_size = 10  # Number of LSTM units in the hidden layer

model = SimpleLSTMModel(hidden_size)
x = torch.randn(50)
f = Functional(model)

In [3]:
print(f.partial_t(x, 0.01))
print(f.partial_x(x, 0.02))
print(f.partial_xx(x, 0.03))


1.1261553
-0.017042458
-0.013079909


In [12]:
path = generate_asset_path(mu = np.array([0.1, 0.2]), sigma = np.array([[0.3, 0.4], [0.1, 0.2]]), T = 1, dt = 0.01)   

In [5]:
# define a class of trading policy 
# the trading policy object

naive_states = np.array([[0.1, 0.2], [0.2, 0.3], [0.3, 0.4], [0.4, 0.5], [0.5, 0.6]])

policy = Trading_Policy(f, naive_states)

In [19]:
# define a simulated trading environment

def simulated_trading(policy, path, intial_wealth, mu, sigma, dt, h, gamma):
    # policy is a trading policy object
    # path is a simulated asset path of dimension (T/dt, n) 
    # n is the number of assets

    # initialize the wealth to be 1
    # meaning the initial cash is 1
    
    # this function is to simulate the trading process
    # return the holding history and wealth history

    wealth_history = np.zeros(len(path[0]))
    holding_history = np.zeros((path.shape[1], len(path)))

    wealth_history[0] = intial_wealth

    for i in range(1, len(path[0])):
        x = wealth_history[:i]
        holding = policy.action(x, mu, sigma, dt, h, gamma)
        holding_history[i] = holding
        
        # to compute the wealth history 
        increment = path[:, i] - path[:, i - 1]
        wealth_change = np.matmul(holding, increment)
        wealth_history[i] = wealth_history[i - 1] + wealth_change

    return holding_history, wealth_history

holding_history, wealth_history = simulated_trading(policy, path, 1, np.array([0.1, 0.2]), np.array([[0.3, 0.4], [0.1, 0.2]]), 0.01, 0.1, 0.1) 

In [20]:

print(wealth_history)

[1.         0.99647592 0.97917434 0.97384783 0.96471388 0.97568628
 0.95097165 0.94003351 0.91467608 1.02003045 1.01733701 0.99466499
 1.00846065 1.05228066 1.03246266 1.02353423 1.01972838 1.01554581
 1.03485273 1.04547821 1.02405383 1.05447598 1.06618436 1.05261621
 1.06694296 1.07283541 1.08189327 1.05276453 1.04963819 1.04150053
 1.00782432 0.975683   0.95128595 0.94683593 0.9490965  0.95680421
 1.00356611 1.01081844 1.00739005 0.99927566 1.02852959 1.03529889
 1.03856027 1.02775846 1.00686643 0.99283443 0.99461546 1.01981952
 1.13429215 1.15138148 1.13617063 1.14341146 1.14004515 1.14316097
 1.18745397 1.18681199 1.19067034 1.22711603 1.24712628 1.25463591
 1.25905043 1.31964695 1.26151781 1.26694969 1.27372084 1.26188876
 1.27568453 1.26915243 1.26984002 1.30875298 1.30355549 1.30822778
 1.29886448 1.29842851 1.2780009  1.26597444 1.2347788  1.27477491
 1.27473989 1.25964085 1.28279611 1.2778761  1.26951751 1.2768669
 1.27931466 1.30474442 1.30948643 1.3117244  1.31658315 1.31631